In [202]:
from pyquery import PyQuery as pq
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json
from pprint import PrettyPrinter as pp

In [12]:
with open('./polinfo.json') as data_file:    
    candidate_json = json.load(data_file)

In [277]:
crimdataset = []

for i,urls in enumerate(candidate_json.keys()):
    page_data = candidate_json[urls]
    if (i%100) == 0:
        print ".",
    crimdataset.append(get_crimdata(2014,page_data,urls))
    
criminal_df = pd.DataFrame(crimdataset)
criminal_df.count()

 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Details                    1400
cognizance_IPC_sections     728
cognizance_Other            728
cognizance_serial_no        728
convicted_IPC_sections       29
convicted_Other              29
convicted_serial_no          29
framed_IPC_sections         821
framed_Other                821
framed_serial_no            821
url                        8204
year_election              8204
dtype: int64

In [269]:
def get_crimdata(year, page_text, url):
    crim_dict = {'year_election': year, 'url': url}
    header = ['serial_no','IPC_sections','Other']
    for h3 in pq(page_text)("h3"):
        if h3.text == "Details of Criminal Cases":
            tablecrims = pq(h3).next('table')
            for h3_2 in pq(tablecrims)("h3"):           
                if h3_2.text == "Cases where charges framed":
                    framed = pq(h3_2).next('table')
                    for i,tr in enumerate(pq(framed)('tr')):
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict["framed_" + header[j]] = child.text

                if h3_2.text == "Cases where Cognizance taken":
                    framed = pq(h3_2).next('table')
                    for i,tr in enumerate(pq(framed)('tr')):
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict["cognizance_" + header[j]] = child.text

                if h3_2.text == "Cases where convicted":
                    framed = pq(h3_2).next('table')
                    for i,tr in enumerate(pq(framed)('tr')):
                        if i > 0:
                            for j,child in enumerate(tr.iterchildren()):
                                if child.text != '---------':
                                    crim_dict["convicted_" + header[j]] = child.text

        if h3.text == "Brief Details of IPCs":
            details = []
            parent = pq(h3).parent()
            for x in pq(parent).children()[1:]:
                dets = pq(x).text()
                if len(dets) > 0:
                    details.append(dets)
                

            crim_dict["Details"] = details
        
    return crim_dict 
        

        

# crim_html = pq(h3)
# crim_html.find
# table = crim_html.next('table')
# print pq(table)


In [270]:
test_index = "candidate.php?candidate_id=8983"
page_data = candidate_json[test_index]
get_crimdata(2014,page_data,test_index)

{'Details': ['1 charges related to Cheating and dishonestly inducing delivery of property (IPC Section-420)'],
 'cognizance_IPC_sections': '309 ',
 'cognizance_Other': ' CC 1111/2010 in Cr No 388/2009 Ehuru II Town L&O PS West Godavari,AP; attempt to commit suicide II addl judicial first class magistrate eluru ',
 'cognizance_serial_no': ' 1',
 'framed_IPC_sections': '420 ',
 'framed_Other': ' CC 398/2013 in crime 104/2013 of Narasapur town PS, West Godavari Dist, Andhra Pradesh;Cheating case; Addl Judicial First Class Magistrate,Narsarpur ',
 'framed_serial_no': ' 1',
 'url': 'candidate.php?candidate_id=8983',
 'year_election': 2014}